In [27]:
%pip install pandas pymongo openpyxl -q


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from pymongo import MongoClient
from pymongo.collection import Collection
from pymongo.database import Database

mongo_client: MongoClient = MongoClient("mongodb://localhost:27017/thesis")
thesis_database: Database = mongo_client["thesis"]
case_collection: Collection = thesis_database["Case"]
evaluation_collection: Collection = thesis_database["Evaluation"]

In [6]:
import sys

sys.path.append("../../src")

In [15]:
from typing import List
from logic.evaluation.model import Evaluation, Metrics


evaluations: List[Evaluation] = [
    Evaluation(
        experiment=e["experiment"],
        model=e["model"],
        version=e["version"],
        date=e["date"],
        metrics=Metrics(**e["metrics"])
    )
    for e in evaluation_collection.find()
]
len(evaluations)

35

In [20]:
from typing import Any, Dict


def normalize_evaluations(evaluations_: List[Evaluation]) -> List[Dict[str, Any]]:
    normalized_evaluations: List[Dict[str, Any]] = []
    for evaluation in evaluations_:
        normalized_evaluations.append({
            "experiment": evaluation.experiment,
            "model": evaluation.model,
            "version": evaluation.version,
            "date": evaluation.date,
            "metric": [k for k in evaluation.metrics.apathetic.keys()][0],
            "apathetic_score": [v for v in evaluation.metrics.apathetic.values()][0],
            "joy_score": [v for v in evaluation.metrics.joy.values()][0],
            "fear_score": [v for v in evaluation.metrics.fear.values()][0],
            "anger_score": [v for v in evaluation.metrics.anger.values()][0],
        })
    return normalized_evaluations

In [23]:
normalized_evaluations: List[Dict[str, Any]] = normalize_evaluations(evaluations_=evaluations)

In [26]:
import pandas as pd

dataframe: pd.DataFrame = pd.DataFrame(normalized_evaluations)
dataframe.sort_values(by=["experiment", "model", "version"], inplace=True)
dataframe.head()

,experiment,model,version,date,metric,apathetic_score,joy_score,fear_score,anger_score
6,BooleanQAExperiment,gemma2,9b-instruct-q4_0,2025-01-19 13:43:34,accuracy,0.882,0.874,0.872,0.866
2,BooleanQAExperiment,llama3.1,8b-instruct-q4_0,2025-01-19 13:43:34,accuracy,0.790,0.794,0.696,0.816
8,BooleanQAExperiment,llama3.2,3b-instruct-q4_0,2025-01-19 13:43:34,accuracy,0.478,0.686,0.640,0.670
0,BooleanQAExperiment,mistral,7b-instruct-q4_0,2025-01-19 13:43:34,accuracy,0.604,0.684,0.724,0.656
4,BooleanQAExperiment,qwen2.5,7b-instruct-q4_0,2025-01-19 13:43:34,accuracy,0.810,0.784,0.730,0.784


In [28]:
dataframe.to_excel("analysis.xlsx")